In [ ]:
import wave
import gc
import librosa
import librosa.display
import IPython
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.client import device_lib

In [ ]:
#configs
tqdm.pandas()

In [ ]:
device_lib.list_local_devices()

In [ ]:
df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")

In [ ]:
df

# Loading and displaying some audio files

In [ ]:
def display_audio_file(audio_file_path):
    y, sr = librosa.load(audio_file_path)
    
    plt.figure(figsize=(14,7))
    
    plt.subplot(2, 2, 1)
    spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(spectrogram, y_axis='linear')
    
    signal_plot = plt.subplot(2, 2, 2)
    signal_plot.plot(y, color="c")
    IPython.display.display(IPython.display.Audio(audio_file_path))


In [ ]:
display_audio_file('../input/urbansound8k/fold8/103076-3-1-0.wav')

In [ ]:
display_audio_file('../input/urbansound8k/fold8/106905-5-0-1.wav')

In [ ]:
display_audio_file('../input/urbansound8k/fold5/100852-0-0-12.wav')

In [ ]:
display_audio_file('../input/urbansound8k/fold2/102871-8-0-1.wav')

# Creating and saving spectrograms in png files

In [ ]:
audio_data_directory = "../input/urbansound8k/"
spectrograms_directory = "/kaggle/temp/spectrograms/"

In [ ]:
def generate_spectrograms(row):

    audio_class = row["classID"]
    spect_directory = "class_" + str(audio_class)
    audio_file_name_without_extension = row["slice_file_name"][:-4]
    
    y, sr = librosa.load(audio_data_directory + "fold" + str(row["fold"]) + "/" + row["slice_file_name"])
    
    spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(spectrogram, y_axis='linear')
    
    plt.savefig(spectrograms_directory + spect_directory + "/" + audio_file_name_without_extension + ".png")

    plt.clf() 
    plt.close('all')
    gc.collect()

In [ ]:
# Creating the classes directories
for i in range(0,10):
    Path(spectrograms_directory + "class_" + str(i)).mkdir(parents=True, exist_ok=True)

In [ ]:
df.progress_apply(generate_spectrograms, axis=1)

# Loading the Dataset

In [ ]:
X_train = tf.keras.preprocessing.image_dataset_from_directory(spectrograms_directory,validation_split = 0.225,
                                                              subset = "training", seed=7)
X_test = tf.keras.preprocessing.image_dataset_from_directory(spectrograms_directory,validation_split = 0.225,
                                                             subset="validation", seed=7)

# Creating and training the model

In [ ]:
model = Sequential([
    
    layers.experimental.preprocessing.Rescaling(1./255,  input_shape=(256, 256, 3)),
    
    layers.Conv2D(32, 7, strides = 4, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    
    layers.MaxPooling2D((4,4)),
    
    layers.Conv2D(128, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),

    layers.Flatten(),
    
    layers.Dense(256),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model_history = model.fit(X_train, validation_data=X_test, epochs=100)

In [ ]:
plt.figure(figsize=(15,5))

plt_loss = plt.subplot(121)
plt_loss.plot(model_history.history["loss"])
plt_loss.plot(model_history.history["val_loss"])
# plt.title("")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="upper right")

plt_accuracy = plt.subplot(122)
plt_accuracy.plot(model_history.history["accuracy"])
plt_accuracy.plot(model_history.history["val_accuracy"])
# plt.title("")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="lower right")

In [ ]:
img = tf.keras.preprocessing.image.load_img("/kaggle/temp/spectrograms/class_3/103076-3-1-0.png", target_size=(256, 256))
img_array = tf.keras.preprocessing.image.img_to_array(img)
print(img_array.shape)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print( "class : "+ str(np.argmax(score)) +" ; Probability = "+ str(np.max(score)))
score